In [2]:
import pandas as pd
import numpy as np
import os
import datetime
from scipy.sparse import hstack, csr_matrix

In [3]:
PATH = '../../../../data/dota2/'
#PATH = '../../../data/dota2/'

In [70]:
df_train_features = pd.read_csv(os.path.join(PATH, 'train_features.csv.zip'), index_col='match_id_hash')
df_test_features = pd.read_csv(os.path.join(PATH, 'test_features.csv.zip'), index_col='match_id_hash')
df_train_targets = pd.read_csv(os.path.join(PATH, 'train_targets.csv.zip'), index_col='match_id_hash')

df_train_features.shape, df_test_features.shape, df_train_targets.shape

((39675, 245), (10000, 245), (39675, 5))

In [72]:
df_train_features = df_train_features.fillna(0)

In [107]:
for col in df_train_features.columns:
    if df_train_features[np.isnan(df_train_features[col])].shape[0] != 0:
        print('There is {0} nan-values in column {1}'.format(df_train_features[np.isnan(df_train_features[col])].shape[0], col))

In [103]:
df_train_features[np.isinf(df_train_features[df_train_features.columns[-1]])]

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,mean_obs_placed_ratio,r_total_sen_placed,d_total_sen_placed,total_sen_placed_ratio,r_std_sen_placed,d_std_sen_placed,std_sen_placed_ratio,r_mean_sen_placed,d_mean_sen_placed,mean_sen_placed_ratio
match_id_hash,,,,,,,,,,,,,,,,,,,,,


In [92]:
df_train_features[np.isinf()]#[['mean_sen_placed_ratio']].info()

ValueError: invalid number of arguments

In [127]:
#df_train_features = df_train_features.replace([np.inf, -np.inf], np.nan)
#df_train_features = df_train_features.fillna(0)
#df_test_features = df_test_features.replace([np.inf, -np.inf], np.nan)
df_test_features = df_test_features.fillna(0)

In [79]:
for c in ['kills', 'deaths', 'assists', 'denies', 'gold', 'lh', 'xp', 'health', 'max_health', 'max_mana', 'level', 'x', 'y', 'stuns', 'creeps_stacked', 'camps_stacked', 'rune_pickups',
          'firstblood_claimed', 'teamfight_participation', 'towers_killed', 'roshans_killed', 'obs_placed', 'sen_placed']:
    r_columns = [f'r{i}_{c}' for i in range(1, 6)]
    d_columns = [f'd{i}_{c}' for i in range(1, 6)]
    
    df_train_features['r_total_' + c] = (df_train_features[r_columns].sum(1)).astype(np.float64)
    df_train_features['d_total_' + c] = (df_train_features[d_columns].sum(1)).astype(np.float64)
    df_train_features['total_' + c + '_ratio'] = (df_train_features['r_total_' + c] / df_train_features['d_total_' + c]).astype(np.float64)
    
    df_test_features['r_total_' + c] = (df_test_features[r_columns].sum(1)).astype(np.float64)
    df_test_features['d_total_' + c] = (df_test_features[d_columns].sum(1)).astype(np.float64)
    df_test_features['total_' + c + '_ratio'] = (df_test_features['r_total_' + c] / df_test_features['d_total_' + c]).astype(np.float64)
    
    df_train_features['r_std_' + c] = (df_train_features[r_columns].std(1)).astype(np.float64)
    df_train_features['d_std_' + c] = (df_train_features[d_columns].std(1)).astype(np.float64)
    df_train_features['std_' + c + '_ratio'] = (df_train_features['r_std_' + c] / df_train_features['d_std_' + c]).astype(np.float64)
    
    df_test_features['r_std_' + c] = (df_test_features[r_columns].std(1)).astype(np.float64)
    df_test_features['d_std_' + c] = (df_test_features[d_columns].std(1)).astype(np.float64)
    df_test_features['std_' + c + '_ratio'] = (df_test_features['r_std_' + c] / df_test_features['d_std_' + c]).astype(np.float64)
    
    df_train_features['r_mean_' + c] = (df_train_features[r_columns].mean(1)).astype(np.float64)
    df_train_features['d_mean_' + c] = (df_train_features[d_columns].mean(1)).astype(np.float64)
    df_train_features['mean_' + c + '_ratio'] = (df_train_features['r_mean_' + c] / df_train_features['d_mean_' + c]).astype(np.float64)
    
    df_test_features['r_mean_' + c] = (df_test_features[r_columns].mean(1)).astype(np.float64)
    df_test_features['d_mean_' + c] = (df_test_features[d_columns].mean(1)).astype(np.float64)
    df_test_features['mean_' + c + '_ratio'] = (df_test_features['r_mean_' + c] / df_test_features['d_mean_' + c]).astype(np.float64)

In [7]:
for feat in df_train_features.columns:
    print(feat)

game_time
game_mode
lobby_type
objectives_len
chat_len
r1_hero_id
r1_kills
r1_deaths
r1_assists
r1_denies
r1_gold
r1_lh
r1_xp
r1_health
r1_max_health
r1_max_mana
r1_level
r1_x
r1_y
r1_stuns
r1_creeps_stacked
r1_camps_stacked
r1_rune_pickups
r1_firstblood_claimed
r1_teamfight_participation
r1_towers_killed
r1_roshans_killed
r1_obs_placed
r1_sen_placed
r2_hero_id
r2_kills
r2_deaths
r2_assists
r2_denies
r2_gold
r2_lh
r2_xp
r2_health
r2_max_health
r2_max_mana
r2_level
r2_x
r2_y
r2_stuns
r2_creeps_stacked
r2_camps_stacked
r2_rune_pickups
r2_firstblood_claimed
r2_teamfight_participation
r2_towers_killed
r2_roshans_killed
r2_obs_placed
r2_sen_placed
r3_hero_id
r3_kills
r3_deaths
r3_assists
r3_denies
r3_gold
r3_lh
r3_xp
r3_health
r3_max_health
r3_max_mana
r3_level
r3_x
r3_y
r3_stuns
r3_creeps_stacked
r3_camps_stacked
r3_rune_pickups
r3_firstblood_claimed
r3_teamfight_participation
r3_towers_killed
r3_roshans_killed
r3_obs_placed
r3_sen_placed
r4_hero_id
r4_kills
r4_deaths
r4_assists
r4_denies


In [65]:
df_train_targets.head()

,game_time,radiant_win,duration,time_remaining,next_roshan_team
match_id_hash,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,False,992,837,NaN
b9c57c450ce74a2af79c9ce96fac144d,658,True,1154,496,NaN
6db558535151ea18ca70a6892197db41,21,True,1503,1482,Radiant
46a0ddce8f7ed2a8d9bd5edcbb925682,576,True,1952,1376,NaN
b1b35ff97723d9b7ade1c9c3cf48f770,453,False,2001,1548,NaN


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler

In [108]:
X = df_train_features.values
y = df_train_targets['radiant_win'].values

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)
len(X_train), len(X_test), y_train.size, y_test.size

(27772, 11903, 27772, 11903)

In [110]:
scaler = StandardScaler()

In [120]:
%%time

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Wall time: 628 ms


In [121]:
X_train_scaled_csr = csr_matrix(X_train_scaled)
X_test_scaled_csr = csr_matrix(X_test_scaled)

In [122]:
%%time

c_values = np.logspace(-2, 1, 20)

model_sc_csr = LogisticRegression(random_state=17, n_jobs=4, solver='liblinear')
logit_grid_searcher = GridSearchCV(estimator=model_sc_csr, param_grid={'C': c_values},
                                       scoring='roc_auc',return_train_score=False, cv=5,
                                       n_jobs=4, verbose=0)
logit_grid_searcher.fit(X_train_scaled_csr, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Wall time: 8min 54s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=4, penalty='l2',
                                          random_state=17, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=4,
             param_grid={'C': array([ 0.01      ,  0.0143845 ,  0.02069138,  0.02976351,  0.04281332,
        0.06158482,  0.08858668,  0.1274275 ,  0.18329807,  0.26366509,
        0.37926902,  0.54555948,  0.78475997,  1.12883789,  1.62377674,
        2.33572147,  3.35981829,  4.83293024,  6.95192796, 10.        ])},
             pre_dispatch

In [123]:
cv_scores = []
for i in range(logit_grid_searcher.n_splits_):
    cv_scores.append(logit_grid_searcher.cv_results_[f'split{i}_test_score'][logit_grid_searcher.best_index_])
print(f'CV scores: {cv_scores}')
print(f'Mean: {np.mean(cv_scores)}, std: {np.std(cv_scores)}\n')

CV scores: [0.8182324038081776, 0.8297694942446692, 0.812195697494593, 0.8175463641014539, 0.8171334442340408]
Mean: 0.8189754807765869, std: 0.005804546917307925



In [124]:
logit_grid_searcher.best_params_

{'C': 0.06158482110660264}

In [128]:
X_valid = df_test_features.values
X_valid_scaled = scaler.transform(X_valid)
X_valid_scaled_csr = csr_matrix(X_valid_scaled)
y_test_pred = logit_grid_searcher.best_estimator_.predict_proba(X_valid_scaled_csr)[:,1]
df_submission = pd.DataFrame({'radiant_win_prob': y_test_pred}, index=df_test_features.index)

submission_filename = 'submission_{}.csv'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
df_submission.to_csv(submission_filename)

print('Submission saved to {}'.format(submission_filename))

Submission saved to submission_2019-11-18_01-09-04.csv


In [117]:
%%time
model_sc_csr = LogisticRegression(random_state=17, n_jobs=4, solver='liblinear')
model_sc_csr.fit(X_train_scaled_csr, y_train)
y_pred = model_sc_csr.predict_proba(X_test_scaled_csr)[:,1]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Wall time: 10.4 s


In [118]:
roc_auc_score(y_test, y_pred)

0.8163824078404676

In [119]:
accuracy_score(y_test, y_pred > 0.5)

0.7292279257330084

#### Без масштабирования результат на тесте очень сильно снижается

In [ ]:
'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'

In [113]:
%%time
model_sc = LogisticRegression(random_state=17, n_jobs=4, C=0.12742749857031335, solver='liblinear')
model_sc.fit(X_train_scaled, y_train)
y_pred = model_sc.predict_proba(X_test_scaled)[:,1]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Wall time: 10.4 s


In [114]:
roc_auc_score(y_test, y_pred)

0.8163990731914487

In [115]:
accuracy_score(y_test, y_pred > 0.5)

0.7298160127698899

In [40]:
X_valid = df_test_features.values
X_valid_scaled = scaler.transform(X_valid)
y_test_pred = model_sc.predict_proba(X_valid_scaled)[:,1]
df_submission = pd.DataFrame({'radiant_win_prob': y_test_pred}, index=df_test_features.index)

submission_filename = 'submission_{}.csv'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
df_submission.to_csv(submission_filename)

print('Submission saved to {}'.format(submission_filename))

Submission saved to submission_2019-11-15_15-02-26.csv


In [76]:
from sklearn.linear_model import LassoCV, RidgeClassifierCV

In [116]:
%%time

lasso = LassoCV(cv=5, random_state=17)

lasso.fit(X_train_scaled, y_train)

y_pred_las = lasso.predict(X_test_scaled)

roc_auc_score(y_test, y_pred_las)

CPU times: user 44.8 s, sys: 12.4 s, total: 57.3 s
Wall time: 15.8 s


/home/anton/jupyter_projects/venv/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1455.0870034545699, tolerance: 0.6924848624513894
  positive)


0.8086353963566544

In [118]:
y_pred_las

array([-0.43565368,  0.46950756,  0.44853005, ...,  0.50135051,
        0.45787696,  0.57789449])

In [117]:
X_valid = df_test_features.values
X_valid_scaled = scaler.transform(X_valid)
y_test_pred = lasso.predict_proba(X_valid_scaled)[:,1]
df_submission = pd.DataFrame({'radiant_win_prob': y_test_pred}, index=df_test_features.index)

submission_filename = 'submission_{}.csv'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
df_submission.to_csv(submission_filename)

print('Submission saved to {}'.format(submission_filename))

AttributeError: 'LassoCV' object has no attribute 'predict_proba'

In [100]:
%%time

ridge = RidgeClassifierCV(cv=10)
ridge.fit(X_train_scaled, y_train)

y_pred_rdg = ridge.predict(X_test_scaled)

roc_auc_score(y_test, y_pred_rdg)

CPU times: user 8.67 s, sys: 3.8 s, total: 12.5 s
Wall time: 3.64 s


0.7193438180743512

In [96]:
def getCoefficients(model, columns):
    """Returns sorted coefficient values of the model"""
    coefs = pd.DataFrame(model.coef_, columns)
    coefs.columns = ["coef"]
    coefs["abs"] = coefs.coef.apply(np.abs)
    return coefs.sort_values(by="abs", ascending=False).drop(["abs"], axis=1)

getCoefficients(lasso, df_train_features.columns)

,coef
d5_level,-0.167303
r1_level,0.162635
d4_level,-0.151502
d3_level,-0.148609
r2_level,0.144113
r4_level,0.144073
d2_level,-0.142363
d1_level,-0.134476
r3_level,0.129554
r5_level,0.124147


In [84]:
lasso.coef_

array([ 2.50745614e-02, -2.01276816e-03,  1.80563030e-03, -1.88526539e-03,
       -2.50471728e-03, -1.43719631e-03,  6.65806834e-03, -6.77375108e-03,
       -1.09466244e-02,  6.62134594e-03,  3.99958854e-02,  0.00000000e+00,
       -5.01199651e-02,  7.44478303e-03,  4.85897637e-03,  4.23568185e-03,
        1.63962012e-01,  5.18740693e-03,  1.01406507e-02,  7.39749402e-03,
        2.96836576e-03,  0.00000000e+00, -0.00000000e+00, -3.45639607e-03,
        1.14520102e-02,  1.09987747e-02, -3.73591768e-03,  2.34599635e-03,
       -4.34910526e-04, -4.54372313e-03,  1.37228326e-02,  6.16953103e-03,
        1.04292335e-02,  1.39498768e-02,  5.93671640e-02,  0.00000000e+00,
       -3.90004089e-02,  7.47066796e-03,  2.78112249e-03, -2.15164501e-03,
        1.43574327e-01,  2.77072752e-03,  7.48216153e-03, -5.02854876e-03,
        8.58137305e-03, -2.86292767e-03,  6.51010535e-03,  3.51478235e-04,
        3.75297673e-03,  7.22322565e-03,  5.77191873e-04,  7.07255432e-03,
        0.00000000e+00, -

In [67]:
from sklearn.ensemble import GradientBoostingClassifier

In [68]:
xgb = GradientBoostingClassifier()

In [69]:
xgb.fit(X_train_scaled, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [71]:
y_pred_xgb = xgb.predict_proba(X_test_scaled)[:,1]
y_pred_xgb

array([0.02008909, 0.44434299, 0.46210541, ..., 0.41915801, 0.69216327,
       0.59271351])

In [72]:
roc_auc_score(y_test, y_pred_xgb)

0.7816539101751729

In [73]:
accuracy_score(y_test, y_pred_xgb > 0.5)

0.7002436360581366

In [74]:
X_valid = df_test_features.values
X_valid_scaled = scaler.transform(X_valid)
y_test_pred = xgb.predict_proba(X_valid_scaled)[:,1]
df_submission = pd.DataFrame({'radiant_win_prob': y_test_pred}, index=df_test_features.index)

submission_filename = 'submission_{}.csv'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
df_submission.to_csv(submission_filename)

print('Submission saved to {}'.format(submission_filename))

Submission saved to submission_2019-11-15_15-40-55.csv
